In [3]:
import sparseml
import sparsezoo
import torch
import torchvision
print(torch.__version__)

1.12.1+cu116


In [4]:
NUM_LABELS = 3
model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
model.fc = torch.nn.Linear(model.fc.in_features, NUM_LABELS)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
import datasets
beans_dataset = datasets.load_dataset("beans")
print(beans_dataset["train"][0]["image_file_path"])
print(beans_dataset["validation"][0]["image_file_path"])

Using custom data configuration default
Reusing dataset beans (/home/ubuntu/.cache/huggingface/datasets/beans/default/0.0.0/d5abfbb94de45599d871182cff389bc870bf145b4829c4b0fe20f0cccd637cbd)


  0%|          | 0/3 [00:00<?, ?it/s]

/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/eeb026374cf5ecfd5f40131a3159be9b9055ac21a3da11690e7eb4d117c99eee/train/bean_rust/bean_rust_train.84.jpg
/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/f287261265d2f9a3e8f87a5526a54d1847b17f7c3ec5714e5719432f2b3e4a73/validation/bean_rust/bean_rust_val.36.jpg


In [6]:
from torchvision import transforms

imagenet_transform = transforms.Compose([
   transforms.Resize(size=256, interpolation=transforms.InterpolationMode.BILINEAR, max_size=None, antialias=None),
   transforms.CenterCrop(size=(224, 224)),
   transforms.ToTensor(),
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(
    root="/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/eeb026374cf5ecfd5f40131a3159be9b9055ac21a3da11690e7eb4d117c99eee/train",
    transform=imagenet_transform
)

val_dataset = torchvision.datasets.ImageFolder(
    root="/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/f287261265d2f9a3e8f87a5526a54d1847b17f7c3ec5714e5719432f2b3e4a73/validation",
    transform=imagenet_transform
)

In [7]:
from tqdm.auto import tqdm
import math

def run_model_one_epoch(model, data_loader, criterion, device, train=False, optimizer=None):
    if train:
        model.train()
    else:
        model.eval()

    running_loss = 0.0
    total_correct = 0
    total_predictions = 0

    # loop through batches
    for step, (inputs, labels) in tqdm(enumerate(data_loader), total=len(data_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        # compute loss, run backpropogation
        outputs = model(inputs)  # model returns logits
        loss = criterion(outputs, labels)
        if train:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()

        # run evaluation
        predictions = outputs.argmax(dim=1)
        total_correct += torch.sum(predictions == labels).item()
        total_predictions += inputs.size(0)

    # return loss and evaluation metric
    loss = running_loss / (step + 1.0)
    accuracy = total_correct / total_predictions
    return loss, accuracy

In [8]:
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

# hyperparameters
BATCH_SIZE = 64

# setup device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Using device: {device}")

# setup data loaders
train_loader = DataLoader(
    train_dataset, BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=16
)
val_loader = DataLoader(
    val_dataset, BATCH_SIZE, shuffle=False, pin_memory=True, num_workers=16
)

# setup loss function and optimizer, LR will be overriden by sparseml
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3)

Using device: cuda


In [9]:
!cat ./dense_model/dense-recipe.yaml


# Epoch and Learning-Rate variables
num_epochs: 10.0
init_lr: 0.0005

training_modifiers:
  - !EpochRangeModifier
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

  - !LearningRateFunctionModifier
    final_lr: 0.0
    init_lr: eval(init_lr)
    lr_func: cosine
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

# Phase 1 Sparse Transfer Learning / Recovery
sparse_transfer_learning_modifiers:
  - !ConstantPruningModifier
    start_epoch: 0.0
    params: __ALL_PRUNABLE__


In [10]:
from sparseml.pytorch.optim import ScheduledModifierManager

# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml("./dense_model/dense-recipe.yaml")
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

In [11]:
from sparseml.pytorch.utils import get_prunable_layers, tensor_sparsity

# print sparsities of each layer
for (name, layer) in get_prunable_layers(model):
    print(f"{name}.weight: {tensor_sparsity(layer.weight).item():.4f}")

conv1.weight: 0.0000
layer1.0.conv1.weight: 0.0000
layer1.0.conv2.weight: 0.0000
layer1.0.conv3.weight: 0.0000
layer1.0.downsample.0.weight: 0.0000
layer1.1.conv1.weight: 0.0000
layer1.1.conv2.weight: 0.0000
layer1.1.conv3.weight: 0.0000
layer1.2.conv1.weight: 0.0000
layer1.2.conv2.weight: 0.0000
layer1.2.conv3.weight: 0.0000
layer2.0.conv1.weight: 0.0000
layer2.0.conv2.weight: 0.0000
layer2.0.conv3.weight: 0.0000
layer2.0.downsample.0.weight: 0.0000
layer2.1.conv1.weight: 0.0000
layer2.1.conv2.weight: 0.0000
layer2.1.conv3.weight: 0.0000
layer2.2.conv1.weight: 0.0000
layer2.2.conv2.weight: 0.0000
layer2.2.conv3.weight: 0.0000
layer2.3.conv1.weight: 0.0000
layer2.3.conv2.weight: 0.0000
layer2.3.conv3.weight: 0.0000
layer3.0.conv1.weight: 0.0000
layer3.0.conv2.weight: 0.0000
layer3.0.conv3.weight: 0.0000
layer3.0.downsample.0.weight: 0.0000
layer3.1.conv1.weight: 0.0000
layer3.1.conv2.weight: 0.0000
layer3.1.conv3.weight: 0.0000
layer3.2.conv1.weight: 0.0000
layer3.2.conv2.weight: 0.000

In [12]:
# run transfer learning
epoch = 0
for epoch in range(manager.max_epochs):
    # run training loop
    epoch_name = f"{epoch + 1}/{manager.max_epochs}"
    print(f"Running Training Epoch {epoch_name}")
    train_loss, train_acc = run_model_one_epoch(
        model, train_loader, criterion, device, train=True, optimizer=optimizer
    )
    print(
        f"Training Epoch: {epoch_name}\nTraining Loss: {train_loss}\nTop 1 Acc: {train_acc}\n"
    )

    # run validation loop
    print(f"Running Validation Epoch {epoch_name}")
    val_loss, val_acc = run_model_one_epoch(model, val_loader, criterion, device)
    print(
        f"Validation Epoch: {epoch_name}\nVal Loss: {val_loss}\nTop 1 Acc: {val_acc}\n"
    )

manager.finalize(model)

Running Training Epoch 1/10


  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 1/10
Training Loss: 0.4339083959074581
Top 1 Acc: 0.8404255319148937

Running Validation Epoch 1/10


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 1/10
Val Loss: 0.248091588417689
Top 1 Acc: 0.9548872180451128

Running Training Epoch 2/10


...

Validation Epoch: 9/10
Val Loss: 0.006565663294168189
Top 1 Acc: 1.0

Running Training Epoch 10/10


  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 10/10
Training Loss: 0.003272765227045645
Top 1 Acc: 1.0

Running Validation Epoch 10/10


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 10/10
Val Loss: 0.007307609868197081
Top 1 Acc: 1.0



In [13]:
from sparseml.pytorch.utils import ModuleExporter

save_dir = "dense_model"
exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="resnet-50-dense-beans.pth")

In [36]:
%cat recipe-0.yaml

# Epoch and Learning-Rate variables
num_epochs: 10.0
init_lr: 0.00025
inter_func: cubic
mask_type: unstructured

training_modifiers:
  - !EpochRangeModifier
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

  - !LearningRateFunctionModifier
    final_lr: 0.0
    init_lr: eval(init_lr)
    lr_func: cosine
    start_epoch: 0.0
    end_epoch: eval(num_epochs)

# Pruning
pruning_modifiers:
   - !GlobalMagnitudePruningModifier
     init_sparsity: 0.05
     final_sparsity: 0.90
     start_epoch: 0.0
     end_epoch: eval(num_epochs)
     update_frequency: 1.0
     params: 
        - 're:layer1.*.conv1.weight'
        - 're:layer1.*.conv2.weight'
        - 're:layer1.*.conv3.weight'
        - 're:layer1.0.downsample.0.weight'
        - 're:layer2.*.conv1.weight'
        - 're:layer2.*.conv2.weight'
        - 're:layer2.*.conv3.weight'
        - 're:layer2.0.downsample.0.weight'
        - 're:layer3.*.conv1.weight'
        - 're:layer3.*.conv2.weight'
        - 're:layer3.*.conv3.weight'
  

In [37]:
checkpoint = torch.load("./dense_model/training/resnet-50-dense-beans.pth")
model = torchvision.models.resnet50()
model.fc = torch.nn.Linear(model.fc.in_features, NUM_LABELS)
model.load_state_dict(checkpoint['state_dict'])
model.to(device)

# setup loss function and optimizer, LR will be overriden by sparseml
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=8e-3)

In [38]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [39]:
# create ScheduledModifierManager and Optimizer wrapper
manager = ScheduledModifierManager.from_yaml("./recipe-0.yaml")
optimizer = manager.modify(model, optimizer, steps_per_epoch=len(train_loader))

In [40]:
for (name, layer) in get_prunable_layers(model):
    print(f"{name}.weight: {tensor_sparsity(layer.weight).item():.4f}")

conv1.weight: 0.0000
layer1.0.conv1.weight: 0.0000
layer1.0.conv2.weight: 0.0000
layer1.0.conv3.weight: 0.0000
layer1.0.downsample.0.weight: 0.0000
layer1.1.conv1.weight: 0.0000
layer1.1.conv2.weight: 0.0000
layer1.1.conv3.weight: 0.0000
layer1.2.conv1.weight: 0.0000
layer1.2.conv2.weight: 0.0000
layer1.2.conv3.weight: 0.0000
layer2.0.conv1.weight: 0.0000
layer2.0.conv2.weight: 0.0000
layer2.0.conv3.weight: 0.0000
layer2.0.downsample.0.weight: 0.0000
layer2.1.conv1.weight: 0.0000
layer2.1.conv2.weight: 0.0000
layer2.1.conv3.weight: 0.0000
layer2.2.conv1.weight: 0.0000
layer2.2.conv2.weight: 0.0000
layer2.2.conv3.weight: 0.0000
layer2.3.conv1.weight: 0.0000
layer2.3.conv2.weight: 0.0000
layer2.3.conv3.weight: 0.0000
layer3.0.conv1.weight: 0.0000
layer3.0.conv2.weight: 0.0000
layer3.0.conv3.weight: 0.0000
layer3.0.downsample.0.weight: 0.0000
layer3.1.conv1.weight: 0.0000
layer3.1.conv2.weight: 0.0000
layer3.1.conv3.weight: 0.0000
layer3.2.conv1.weight: 0.0000
layer3.2.conv2.weight: 0.000

In [41]:
# run transfer learning
epoch = 0
for epoch in range(manager.max_epochs):
    # run training loop
    epoch_name = f"{epoch + 1}/{manager.max_epochs}"
    print(f"Running Training Epoch {epoch_name}")
    train_loss, train_acc = run_model_one_epoch(
        model, train_loader, criterion, device, train=True, optimizer=optimizer
    )
    print(
        f"Training Epoch: {epoch_name}\nTraining Loss: {train_loss}\nTop 1 Acc: {train_acc}\n"
    )

    # run validation loop
    print(f"Running Validation Epoch {epoch_name}")
    val_loss, val_acc = run_model_one_epoch(model, val_loader, criterion, device)
    print(
        f"Validation Epoch: {epoch_name}\nVal Loss: {val_loss}\nTop 1 Acc: {val_acc}\n"
    )
    
    print(f"\n\nChecking Sparsity Level:")
    for (name, layer) in get_prunable_layers(model):
        print(f"{name}.weight: {tensor_sparsity(layer.weight).item():.4f}")
    print(f"\n")
    
manager.finalize(model)

Running Training Epoch 1/10


  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 1/10
Training Loss: 0.009745531282319194
Top 1 Acc: 0.9980657640232108

Running Validation Epoch 1/10


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 1/10
Val Loss: 0.03642222502579292
Top 1 Acc: 0.9849624060150376



Checking Sparsity Level:
conv1.weight: 0.0000
layer1.0.conv1.weight: 0.0156
layer1.0.conv2.weight: 0.0299
layer1.0.conv3.weight: 0.0468
layer1.0.downsample.0.weight: 0.0452
layer1.1.conv1.weight: 0.0623
layer1.1.conv2.weight: 0.0414
layer1.1.conv3.weight: 0.0328
layer1.2.conv1.weight: 0.0204
layer1.2.conv2.weight: 0.0228
layer1.2.conv3.weight: 0.0385
layer2.0.conv1.weight: 0.0201
layer2.0.conv2.weight: 0.0337
layer2.0.conv3.weight: 0.0354
layer2.0.downsample.0.weight: 0.0452
layer2.1.conv1.weight: 0.0430
layer2.1.conv2.weight: 0.0410
layer2.1.conv3.weight: 0.0340
layer2.2.conv1.weight: 0.0350
layer2.2.conv2.weight: 0.0398
layer2.2.conv3.weight: 0.0384
layer2.3.conv1.weight: 0.0297
layer2.3.conv2.weight: 0.0348
layer2.3.conv3.weight: 0.0418
layer3.0.conv1.weight: 0.0280
layer3.0.conv2.weight: 0.0441
layer3.0.conv3.weight: 0.0345
layer3.0.downsample.0.weight: 0.0551
layer3.1.conv1.weight: 0.0492
layer3.

...

Validation Epoch: 9/10
Val Loss: 0.046517357851068176
Top 1 Acc: 1.0



Checking Sparsity Level:
conv1.weight: 0.0000
layer1.0.conv1.weight: 0.4712
layer1.0.conv2.weight: 0.6728
layer1.0.conv3.weight: 0.5400
layer1.0.downsample.0.weight: 0.5073
layer1.1.conv1.weight: 0.5588
layer1.1.conv2.weight: 0.6290
layer1.1.conv3.weight: 0.5516
layer1.2.conv1.weight: 0.5294
layer1.2.conv2.weight: 0.5957
layer1.2.conv3.weight: 0.5948
layer2.0.conv1.weight: 0.5384
layer2.0.conv2.weight: 0.7441
layer2.0.conv3.weight: 0.6780
layer2.0.downsample.0.weight: 0.8222
layer2.1.conv1.weight: 0.7831
layer2.1.conv2.weight: 0.8004
layer2.1.conv3.weight: 0.7358
layer2.2.conv1.weight: 0.7461
layer2.2.conv2.weight: 0.8004
layer2.2.conv3.weight: 0.7619
layer2.3.conv1.weight: 0.7125
layer2.3.conv2.weight: 0.7751
layer2.3.conv3.weight: 0.7315
layer3.0.conv1.weight: 0.6695
layer3.0.conv2.weight: 0.8614
layer3.0.conv3.weight: 0.7442
layer3.0.downsample.0.weight: 0.9151
layer3.1.conv1.weight: 0.8794
layer3.1.conv2.weight

  0%|          | 0/17 [00:00<?, ?it/s]

Training Epoch: 10/10
Training Loss: 0.032790856435894966
Top 1 Acc: 1.0

Running Validation Epoch 10/10


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Epoch: 10/10
Val Loss: 0.0411220183596015
Top 1 Acc: 1.0



Checking Sparsity Level:
conv1.weight: 0.0000
layer1.0.conv1.weight: 0.4783
layer1.0.conv2.weight: 0.6819
layer1.0.conv3.weight: 0.5481
layer1.0.downsample.0.weight: 0.5146
layer1.1.conv1.weight: 0.5688
layer1.1.conv2.weight: 0.6389
layer1.1.conv3.weight: 0.5606
layer1.2.conv1.weight: 0.5388
layer1.2.conv2.weight: 0.6055
layer1.2.conv3.weight: 0.6023
layer2.0.conv1.weight: 0.5475
layer2.0.conv2.weight: 0.7528
layer2.0.conv3.weight: 0.6869
layer2.0.downsample.0.weight: 0.8294
layer2.1.conv1.weight: 0.7904
layer2.1.conv2.weight: 0.8076
layer2.1.conv3.weight: 0.7448
layer2.2.conv1.weight: 0.7550
layer2.2.conv2.weight: 0.8084
layer2.2.conv3.weight: 0.7696
layer2.3.conv1.weight: 0.7220
layer2.3.conv2.weight: 0.7845
layer2.3.conv3.weight: 0.7395
layer3.0.conv1.weight: 0.6785
layer3.0.conv2.weight: 0.8685
layer3.0.conv3.weight: 0.7536
layer3.0.downsample.0.weight: 0.9200
layer3.1.conv1.weight: 0.8859
layer3.1.conv2.weight:

In [42]:
save_dir = "experiment-0"
exporter = ModuleExporter(model, output_dir=save_dir)
exporter.export_pytorch(name="resnet-50-sparse-experiment-0.pth")